In [1]:
# this sets up basic packages
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.cosmology.units as cu
import h5py

# this sets up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# this sets up astropy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky, Distance
from astropy.table import Table, unique, setdiff

from regions import Regions, CircleSkyRegion

In [2]:
dream = fits.open('/Users/ngbinh/Downloads/DREaM_main.fits')
dream_data = Table(dream[1].data)

In [3]:
# this selects all the neighbors with 2 < z < 5, then gets their Mstar and SFR
relevant = dream_data['ID', 'RA', 'Dec', 'redshift', 'M_gal', 'logpsi'][dream_data['redshift']<5]
relevant = relevant[relevant['redshift']>2].to_pandas()

In [4]:
# this selects all the ones that are quenched (i.e. sSFR < -9.8)
quenched_ones = relevant[np.log10(10**relevant['logpsi'] / relevant['M_gal']) < -9.8]
quenched_ones = quenched_ones[quenched_ones['M_gal'] > 10**10.5]

In [5]:
relevant

,ID,RA,Dec,redshift,M_gal,logpsi
0,13241306,0.489857,0.010564,2.014664,1.273912e+07,-1.369086
1,13241307,0.489621,0.015140,2.014622,1.961021e+05,-2.058861
2,13241308,0.490752,-0.043852,2.014522,7.534733e+05,-2.632520
3,13241309,0.490896,-0.043992,2.014489,2.077482e+05,1.622524
4,13241310,0.492010,-0.043814,2.014393,1.295432e+05,-3.098835
...,...,...,...,...,...,...
12343026,25951780,-0.395015,0.470712,4.977378,1.483325e+05,1.476693
12343027,25951781,-0.325127,0.410665,4.984012,3.898026e+05,-1.798956
12343028,25951782,-0.052119,-0.499995,4.966891,2.134280e+05,-2.114437
12343029,25951783,0.170450,-0.499948,4.950807,2.765631e+06,-1.781149


There are too many quenched galaxies in this catalog, so let's just randomly sample 100 of them.

In [7]:
# this selects all the ones that are NOT quenched in relevant. the potential neighbors will be in here.
others = pd.concat([relevant, quenched_ones]).drop_duplicates(keep=False)

In [8]:
quenched_sample = quenched_ones.sample(n=100)

In [9]:
# this creates SkyCoord objects for the quenched sample and the non-quenched ones
quenched_cat = SkyCoord(ra=quenched_sample['RA'].values*u.degree, dec=quenched_sample['Dec'].values*u.degree)
others_cat = SkyCoord(ra=others['RA'].values*u.degree, dec=others['RA'].values*u.degree)

In [10]:
neighbor_numbers = []

# this looks for neighbors for each of the quenched ones within 15 arcsec
for quenched_object in quenched_cat:
    
    # use separation to find nearby objects ("nearby" in terms of RA and Dec) in the catalog
    d2d = quenched_object.separation(others_cat)
    catalogmsk = d2d < 15*u.arcsec
    neighbors_ra = d2d[catalogmsk]# others['RA'].values[catalogmsk]
    print(np.size(neighbors_ra))

    neighbor_numbers.append(np.size(neighbors_ra))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
51398
0
0
0
0
0
0
0
0
0
65804
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [11]:
neighbor_numbers

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 51398,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 65804,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]